In [1]:
%load_ext autoreload
%autoreload 2

import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

import time

from Analysis import *

In [2]:
file = uproot.open("../build/output.root")
enteringTree = file['Entering']
leavingTree = file['Leaving']
recoilsTree = file['Recoils']
edepTree = file['Edep']
entering = enteringTree.arrays(library="pd")
leaving = leavingTree.arrays(library="pd")
recoils = recoilsTree.arrays(library="pd")
edep = edepTree.arrays(library="pd")

In [3]:
leavingCounts = CountsThroughSurface(leaving)
enteringCounts = CountsThroughSurface(entering)

In [4]:
data = []

for eventID in recoils['Event ID'].value_counts().keys():
    events = recoils[(recoils['Event ID'] == eventID)]
    eventTrueRecoils = events[~events['Parent ID'].isin(events['Track ID'])]
    
    for index, row in eventTrueRecoils.iterrows():
        data.append(row)

trueRecoils = (
    pd.DataFrame(data, columns=recoils.dtypes.index)
    .astype(dtype = recoils.dtypes)
)

display(trueRecoils)

,Event ID,Track ID,Parent ID,Global Time (ns),Particle Name,Energy (keV),Position x (mm),Position y (mm),Position z (mm)
117,3976,6,4,50.528471,e-,1126.976409,4.156806,11.133155,12.179090
184,3976,145,78,50.691733,e-,0.635460,-30.885403,28.353027,10.370628
197,3976,148,66,50.665561,e-,0.255890,-26.208834,27.137317,5.551792
198,3976,147,66,50.665561,e-,0.380310,-26.208834,27.137317,5.551792
261,3976,3,1,50.516076,F19,250.633759,7.453504,9.703110,13.125369
...,...,...,...,...,...,...,...,...,...
266,4055,3,1,204.672872,F19,388.043069,-2.716204,16.476062,-44.948521
267,4121,2,1,85.550615,F19,134.039360,-17.891384,0.036027,-55.322821
268,4138,2,1,-118.615705,F19,196.813641,4.009611,28.604532,-66.510336
269,4142,2,1,-186.129248,F19,3.241926,16.270386,-6.217385,-73.171612


In [5]:
trueRecoils.insert(trueRecoils.shape[1], 'End Position x (mm)', np.nan)
trueRecoils.insert(trueRecoils.shape[1], 'End Position y (mm)', np.nan)
trueRecoils.insert(trueRecoils.shape[1], 'End Position z (mm)', np.nan)

In [6]:
for index, row in trueRecoils.iterrows():
    eventID = row['Event ID']
    trackID = row['Track ID']
    
    events = edep[(edep['Event ID'] == eventID) & (edep['Track ID'] == trackID)]
    lastStep = events.loc[events['Global Time (ns)'].idxmax()]
    
    trueRecoils.loc[index, 'End Position x (mm)'] = lastStep['End Position x (mm)']
    trueRecoils.loc[index, 'End Position y (mm)'] = lastStep['End Position y (mm)']
    trueRecoils.loc[index, 'End Position z (mm)'] = lastStep['End Position z (mm)']

display(trueRecoils)

,Event ID,Track ID,Parent ID,Global Time (ns),Particle Name,Energy (keV),Position x (mm),Position y (mm),Position z (mm),End Position x (mm),End Position y (mm),End Position z (mm)
117,3976,6,4,50.528471,e-,1126.976409,4.156806,11.133155,12.179090,-36.372702,37.184074,13.090514
184,3976,145,78,50.691733,e-,0.635460,-30.885403,28.353027,10.370628,-30.885486,28.352891,10.370647
197,3976,148,66,50.665561,e-,0.255890,-26.208834,27.137317,5.551792,-26.208866,27.137320,5.551761
198,3976,147,66,50.665561,e-,0.380310,-26.208834,27.137317,5.551792,-26.208808,27.137362,5.551844
261,3976,3,1,50.516076,F19,250.633759,7.453504,9.703110,13.125369,7.454369,9.700064,13.115843
...,...,...,...,...,...,...,...,...,...,...,...,...
266,4055,3,1,204.672872,F19,388.043069,-2.716204,16.476062,-44.948521,-2.717481,16.471114,-44.960977
267,4121,2,1,85.550615,F19,134.039360,-17.891384,0.036027,-55.322821,-17.895223,0.032573,-55.326025
268,4138,2,1,-118.615705,F19,196.813641,4.009611,28.604532,-66.510336,4.011847,28.610592,-66.515522
269,4142,2,1,-186.129248,F19,3.241926,16.270386,-6.217385,-73.171612,16.270432,-6.217372,-73.171616


In [7]:
minx = min(trueRecoils['Position x (mm)'].min(), trueRecoils['End Position x (mm)'].min())
miny = min(trueRecoils['Position y (mm)'].min(), trueRecoils['End Position y (mm)'].min())
minz = min(trueRecoils['Position z (mm)'].min(), trueRecoils['End Position z (mm)'].min())
mint = trueRecoils['Global Time (ns)'].min()

maxx = max(trueRecoils['Position x (mm)'].max(), trueRecoils['End Position x (mm)'].max())
maxy = max(trueRecoils['Position y (mm)'].max(), trueRecoils['End Position y (mm)'].max())
maxz = max(trueRecoils['Position z (mm)'].max(), trueRecoils['End Position z (mm)'].max())
maxt = trueRecoils['Global Time (ns)'].max()

In [8]:
trueRecoils.insert(trueRecoils.shape[1], 'Bin xi', 0)
trueRecoils.insert(trueRecoils.shape[1], 'Bin yi', 0)
trueRecoils.insert(trueRecoils.shape[1], 'Bin zi', 0)
trueRecoils.insert(trueRecoils.shape[1], 'Bin xf', 0)
trueRecoils.insert(trueRecoils.shape[1], 'Bin yf', 0)
trueRecoils.insert(trueRecoils.shape[1], 'Bin zf', 0)

trueRecoils.insert(trueRecoils.shape[1], 'Bin t', 0)

In [19]:
spatialResolution = 1 # in mm      (Purely me guessing)
temporalResolution = 20000 # in ns (Steve said it might be 20 us)

In [20]:
trueRecoils['Bin xi'] = np.floor((trueRecoils['Position x (mm)'] - minx) / spatialResolution).astype(np.int32)
trueRecoils['Bin yi'] = np.floor((trueRecoils['Position y (mm)'] - miny) / spatialResolution).astype(np.int32)
trueRecoils['Bin zi'] = np.floor((trueRecoils['Position z (mm)'] - minz) / spatialResolution).astype(np.int32)
trueRecoils['Bin xf'] = np.floor((trueRecoils['End Position x (mm)'] - minx) / spatialResolution).astype(np.int32)
trueRecoils['Bin yf'] = np.floor((trueRecoils['End Position y (mm)'] - miny) / spatialResolution).astype(np.int32)
trueRecoils['Bin zf'] = np.floor((trueRecoils['End Position z (mm)'] - minz) / spatialResolution).astype(np.int32)

trueRecoils['Bin t'] = np.floor((trueRecoils['Global Time (ns)'] - mint) / temporalResolution).astype(np.int32)

In [21]:
trueRecoils

,Event ID,Track ID,Parent ID,Global Time (ns),Particle Name,Energy (keV),Position x (mm),Position y (mm),Position z (mm),End Position x (mm),End Position y (mm),End Position z (mm),Bin xi,Bin yi,Bin zi,Bin xf,Bin yf,Bin zf,Bin t
117,3976,6,4,50.528471,e-,1126.976409,4.156806,11.133155,12.179090,-36.372702,37.184074,13.090514,95,102,111,54,128,112,0
184,3976,145,78,50.691733,e-,0.635460,-30.885403,28.353027,10.370628,-30.885486,28.352891,10.370647,60,119,109,60,119,109,0
197,3976,148,66,50.665561,e-,0.255890,-26.208834,27.137317,5.551792,-26.208866,27.137320,5.551761,64,118,104,64,118,104,0
198,3976,147,66,50.665561,e-,0.380310,-26.208834,27.137317,5.551792,-26.208808,27.137362,5.551844,64,118,104,64,118,104,0
261,3976,3,1,50.516076,F19,250.633759,7.453504,9.703110,13.125369,7.454369,9.700064,13.115843,98,101,112,98,101,112,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,4055,3,1,204.672872,F19,388.043069,-2.716204,16.476062,-44.948521,-2.717481,16.471114,-44.960977,88,108,54,88,108,54,0
267,4121,2,1,85.550615,F19,134.039360,-17.891384,0.036027,-55.322821,-17.895223,0.032573,-55.326025,73,91,43,73,91,43,0
268,4138,2,1,-118.615705,F19,196.813641,4.009611,28.604532,-66.510336,4.011847,28.610592,-66.515522,94,120,32,94,120,32,0
269,4142,2,1,-186.129248,F19,3.241926,16.270386,-6.217385,-73.171612,16.270432,-6.217372,-73.171616,107,85,26,107,85,26,0


In [58]:
falsePositives = pd.DataFrame(None, columns=['Event ID 1', 'Track ID 1', 'Event ID 2', 'Track ID 2', 'Distance'])
falsePositives = falsePositives.astype(dtype={'Event ID 1':'int32', 'Event ID 2':'int32','Track ID 1':'int32', 'Track ID 2':'int32','Distance':'float64'})

for index, row in trueRecoils.iterrows():
    possibleFalsePositives = trueRecoils[
        ((row['Track ID'] != trueRecoils['Track ID']) | (row['Event ID'] != trueRecoils['Event ID'])) &
        trueRecoils['Bin xi'].between(row['Bin xi'] - 1, row['Bin xi'] + 1) &
        trueRecoils['Bin yi'].between(row['Bin yi'] - 1, row['Bin yi'] + 1) &
        trueRecoils['Bin zi'].between(row['Bin zi'] - 1, row['Bin zi'] + 1)
    ]
    
    for index2, row2 in possibleFalsePositives.iterrows():
        # First check to see if we have already had this event but backwards
        if ((falsePositives[
                (falsePositives['Event ID 1'] == row2['Event ID']) &
                (falsePositives['Event ID 2'] == row['Event ID']) &
                (falsePositives['Track ID 1'] == row2['Track ID']) &
                (falsePositives['Track ID 2'] == row['Track ID'])
            ]).shape[0] > 0):
            continue
        xa = row['Position x (mm)']
        ya = row['Position y (mm)']
        za = row['Position z (mm)']
        
        xb = row2['Position x (mm)']
        yb = row2['Position y (mm)']
        zb = row2['Position z (mm)']
        
        dist = np.sqrt((xa - xb) ** 2 + (ya - yb) ** 2 + (za - zb) ** 2)
        
        if (dist < spatialResolution):
            falsePositives = pd.concat([falsePositives, pd.DataFrame([[row['Event ID'], row['Track ID'], row2['Event ID'], row2['Track ID'], dist]], columns = falsePositives.columns)], ignore_index=True)

In [61]:
falsePositives

,Event ID 1,Track ID 1,Event ID 2,Track ID 2,Distance
0,3976,148,3976,147,0.000000
1,6408,26,6408,25,0.000000
2,9713,6,9713,5,0.000000
3,8948,9,8948,8,0.000000
4,8948,9,8948,11,0.051921
5,8948,9,8948,10,0.051921
6,8948,9,8948,3,0.030659
7,8948,8,8948,11,0.051921
8,8948,8,8948,10,0.051921
9,8948,8,8948,3,0.030659
